In [1]:
!pip install openai

import openai

In [2]:
import openai
import json
import numpy as np
import time
import pickle
from tqdm import tqdm

def parse_bullets(sentence):
    bullets_preprocess = sentence.split("\n")
    bullets = []

    for bullet in bullets_preprocess:
        try:
            idx = bullet.find(next(filter(str.isalpha, bullet)))
        except:
            continue

        bullet = bullet[idx:]

        if len(bullet) != 0:
            bullets.append(bullet)

    return bullets


def generate_answer(answer_context):
    try:

        completion = openai.chat.completions.create(
                  model="gpt-3.5-turbo",
                  messages=answer_context,
                  n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(answer_context)

    return completion


def construct_message(agents, question, idx):

    # Use introspection in the case in which there are no other agents.
    if len(agents) == 0:
        return {"role": "user", "content": "Can you verify that your answer is correct. Please reiterate your answer, making sure to state your answer at the end of the response."}

    prefix_string = "These are the recent/updated opinions from other agents: "

    for agent in agents:
        agent_response = agent[idx]["content"]
        response = "\n\n One agent response: ```{}```".format(agent_response)

        prefix_string = prefix_string + response

    prefix_string = prefix_string + "\n\n Use these opinions carefully as additional advice, can you provide an updated answer? Make sure to state your answer at the end of the response.".format(question)
    return {"role": "user", "content": prefix_string}


def construct_assistant_message(completion):
    content = completion.choices[0].message.content
    return {"role": "assistant", "content": content}

def parse_answer(sentence):
    parts = sentence.split(" ")

    for part in parts[::-1]:
        try:
            answer = float(part)
            return answer
        except:
            continue


def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        current_frequency = List.count(i)
        if current_frequency > counter:
            counter = current_frequency
            num = i

    return num


if __name__ == "__main__":
    answer = parse_answer("My answer is the same as the other agents and AI language model: the result of 12+28*19+6 is 550.")

    agents = 2
    rounds = 3
    np.random.seed(0)

    evaluation_round = 100
    scores = []

    generated_description = {}

    for round in tqdm(range(evaluation_round)):
        a, b, c, d, e, f = np.random.randint(0, 30, size=6)

        answer = a + b * c + d - e * f
        agent_contexts = [[{"role": "user", "content": """What is the result of {}+{}*{}+{}-{}*{}? Make sure to state your answer at the end of the response.""".format(a, b, c, d, e, f)}] for agent in range(agents)]

        content = agent_contexts[0][0]['content']
        question_prompt = "We seek to find the result of {}+{}*{}+{}-{}*{}?".format(a, b, c, d, e, f)

        for round in range(rounds):
            for i, agent_context in enumerate(agent_contexts):

                if round != 0:
                    agent_contexts_other = agent_contexts[:i] + agent_contexts[i+1:]
                    message = construct_message(agent_contexts_other, question_prompt, 2*round - 1)
                    agent_context.append(message)

                    print("message: ", message)

                completion = generate_answer(agent_context)

                assistant_message = construct_assistant_message(completion)
                agent_context.append(assistant_message)
                print(completion)

        text_answers = []

        for agent_context in agent_contexts:
            text_answer = string =  agent_context[-1]['content']
            text_answer = text_answer.replace(",", ".")
            text_answer = parse_answer(text_answer)

            if text_answer is None:
                continue

            text_answers.append(text_answer)

        generated_description[(a, b, c, d, e, f)] = (agent_contexts, answer)

        try:
            text_answer = most_frequent(text_answers)
            if text_answer == answer:
                scores.append(1)
            else:
                scores.append(0)
        except:
            continue

        print("performance:", np.mean(scores), np.std(scores) / (len(scores) ** 0.5))

    pickle.dump(generated_description, open("math_agents{}_rounds{}.p".format(agents, rounds), "wb"))
    import pdb
    pdb.set_trace()
    print(answer)
    print(agent_context)

  0%|          | 0/100 [00:00<?, ?it/s]

ChatCompletion(id='chatcmpl-BBleoPzxxYnTpozW5noAVygcNgM1e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To solve this expression, we need to follow the order of operations which is parentheses, exponents, multiplication and division (from left to right), addition and subtraction (from left to right).\n\n12 + 15 * 21 + 0 - 3 * 27\n= 12 + 315 + 0 - 81\n= 327 - 81\n= 246\n\nTherefore, the result of 12 + 15 * 21 + 0 - 3 * 27 is 246.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1742144502, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=106, prompt_tokens=38, total_tokens=144, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0,

  1%|          | 1/100 [00:08<14:34,  8.83s/it]

ChatCompletion(id='chatcmpl-BBlevxpuVvE4EblYvkBzfVkZ4L0VE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='After reviewing the input from the other agents and reassessing the expression, the correct result was determined to be 246. Therefore, the final answer is:\n\n12 + 15 * 21 + 0 - 3 * 27 = 246', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1742144509, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=52, prompt_tokens=639, total_tokens=691, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
performance: 1.0 0.0
ChatCompletion(id='chatcmpl-BBlew5cg1J1JlcqB0BUUD8QGuBh6I', choices=[Choice(finish_reason='stop'

  2%|▏         | 2/100 [00:17<14:26,  8.85s/it]

...

ChatCompletion(id='chatcmpl-BBlsAW9F6ZVntyrYzamBmGsYmUQyq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Taking into account the valid explanation provided by the other agents, we apply the order of operations (PEMDAS) in solving the given expression:\n\n4 + 26*1 + 17 - 0*7\n= 4 + 26 + 17 - 0\n= 30 + 17 - 0\n= 47 - 0\n= 47\n\nTherefore, the result of the expression 4 + 26*1 + 17 - 0*7 is 47.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1742145330, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=105, prompt_tokens=600, total_tokens=705, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
performance: 0.919

100%|██████████| 100/100 [14:00<00:00,  8.40s/it]

PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.11/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



ChatCompletion(id='chatcmpl-BBlsJcyFFVF1NHcfYvBE1UMXkgEi9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Following the order of operations:\n\n24 + 26 * 4 + 3 - 5 * 6\n\n1. Perform multiplication first:\n   = 24 + 104 + 3 - 30\n\n2. Perform addition and subtraction from left to right:\n   = 128 + 3 - 30\n   = 131 - 30\n\n3. Finally, subtract:\n   = 101\n\nTherefore, after correctly following the order of operations, the result of 24 + 26 * 4 + 3 - 5 * 6 is 101.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1742145339, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=119, prompt_tokens=632, total_tokens=751, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=Pr


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.11/bdb.py", line 361, in set_quit
    sys.settrace(None)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/debugger.py", line 1075, in cmdloop
    sys.settrace(None)



--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
101
[{'role': 'user', 'content': 'What is the result of 24+26*4+3-5*6? Make sure to state your answer at the end of the response.'}, {'role': 'assistant', 'content': 'To solve the equation, we must follow the order of operations (PEMDAS):\n\n24 + 26 * 4 + 3 - 5 * 6\n= 24 + 104 + 3 - 30\n= 131 + 3 - 30\n= 134 - 30\n= 104\n\nTherefore, the result of 24 + 26 * 4 + 3 - 5 * 6 is 104.'}, {'role': 'user', 'content': "These are the recent/updated opinions from other agents: \n\n One agent response: ```To solve this equation, we must follow the order of operations (PEMDAS):\n\n1. Parentheses (none in this case)\n2. Exponents (none in this case)\n3. Multiplication and Division (from left to right)\n4. Addition and Subtraction (from left to right)\n\nNow, let's solve it step by step:\n\n24 + 26 * 4 + 3 - 5 * 6\n= 24 + 104 + 3 - 30\n= 128 + 3 - 30\n= 131 - 30\n= 101\n\nTherefore, the result of 24 + 26 * 4 + 3 - 5 * 6 is 101.```\n\n Use 

In [3]:
pickle.dump(generated_description, open("math_agents{}_rounds{}.pk".format(agents, rounds), "wb"))

In [8]:
# prompt: read from pk file to dataframe and store in csv

import pandas as pd
import pickle

# Load the data from the pickle file
with open("math_agents2_rounds3.pk", "rb") as f:
    data = pickle.load(f)

# Convert the data to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Reset the index to make the original index a column
df = df.reset_index()

# Rename columns for clarity (optional)
df.columns = ['input_parameters', 'agent_contexts', 'expected_answer']

# Separate the input parameters tuple into separate columns
df[['a', 'b', 'c', 'd', 'e', 'f']] = pd.DataFrame(df['input_parameters'].tolist(), index= df.index)

# Save the DataFrame to a CSV file
df.to_csv('math_results.csv', index=False)


In [6]:
print(answer)
print(agent_context)

101
[{'role': 'user', 'content': 'What is the result of 24+26*4+3-5*6? Make sure to state your answer at the end of the response.'}, {'role': 'assistant', 'content': 'To solve the equation, we must follow the order of operations (PEMDAS):\n\n24 + 26 * 4 + 3 - 5 * 6\n= 24 + 104 + 3 - 30\n= 131 + 3 - 30\n= 134 - 30\n= 104\n\nTherefore, the result of 24 + 26 * 4 + 3 - 5 * 6 is 104.'}, {'role': 'user', 'content': "These are the recent/updated opinions from other agents: \n\n One agent response: ```To solve this equation, we must follow the order of operations (PEMDAS):\n\n1. Parentheses (none in this case)\n2. Exponents (none in this case)\n3. Multiplication and Division (from left to right)\n4. Addition and Subtraction (from left to right)\n\nNow, let's solve it step by step:\n\n24 + 26 * 4 + 3 - 5 * 6\n= 24 + 104 + 3 - 30\n= 128 + 3 - 30\n= 131 - 30\n= 101\n\nTherefore, the result of 24 + 26 * 4 + 3 - 5 * 6 is 101.```\n\n Use these opinions carefully as additional advice, can you provide